### Code converts model saved in .h5 file to file in .tflite or .lite format
### Also code quantize theese recieved models

In [0]:
#Importing beta version of TensorFlow 2.0
!pip install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow_hub

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from google.colab import files

In [0]:
import tensorflow as tf
from keras.models import Model

#Indicate a path to your saved .h5 model with transfer learning
model = "/content/gdrive/My Drive/pdd_classifier_all_150epochs_100epochs.h5"

#This folder would keep .pb and others files with your model
SAVED_MODEL = "/content/gdrive/My Drive/pdd_model_new/pdd/"
tf.keras.experimental.export_saved_model(tf.keras.models.load_model(model), SAVED_MODEL)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [0]:
model = tf.keras.experimental.load_from_saved_model(SAVED_MODEL)

In [0]:
#Write a path to folder where would be saved all files regarding your model
!mkdir "/content/gdrive/My Drive/pdd_model_new/tfmodels/"

In [0]:
#Indicate a file in neccessary format where non-quantized model would be saved
TFLITE_MODEL = "/content/gdrive/My Drive/pdd_model_new/tfmodels/model.tflite"
#Indicate a file in neccessary format where quantized model would be saved
TFLITE_QUANT_MODEL = "/content/gdrive/My Drive/pdd_model_new/tfmodels/model_quant.tflite"

In [0]:
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

# Convert the model to quantized version with post-training quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
open(TFLITE_QUANT_MODEL, "wb").write(tflite_quant_model)

print("TFLite models and their sizes:")
!ls "/content/gdrive/My Drive/pdd_model_new/tfmodels/" -lh